In [2]:
import importlib
import e7awgsw
import bokeh.io
import bokeh.plotting
import bokeh.models
import qubelsi.qube
import pickle
from IPython.display import clear_output

import qubeui
import instruments

bokeh.plotting.output_notebook()

QUBE = 'Riken1'
QUBE_ID = 4

qube = qubelsi.qube.Qube(
    '10.5.0.22', # IP address of eXtickGE
    './adi_api_mod', # Path to API
)
qubeui.StartupUI(qube)
visa = instruments.new_resource_manager()

Loading BokehJS ...

Text(value='10.5.0.22', description='IP Address for eXtickGE', disabled=True, style=DescriptionStyle(descripti…

Text(value='./adi_api_mod', description='Path to API', disabled=True, layout=Layout(width='50%'), style=Descri…

Button(description='Do init', layout=Layout(height='80px', width='50%'), style=ButtonStyle())

Button(description='Restart AD9082', layout=Layout(height='80px', width='50%'), style=ButtonStyle())

In [5]:
importlib.reload(qubeui)
qubeui.AwgPanel(qube).display()

In [6]:
port = qubeui.new_port_handler(qube)
qubeui.set_flo_fnco_according_to_frf(port[5], 8500, 1900)

(10400, 1897.5)

# ↑ は初期化コードですので普段は使わないでください

---

# 周波数特性測定コード

In [ ]:
#r = ms2038.measure()
# ADRF6780のVATTを変更しながらスペクトルを取得する
import pickle

PORT_ID = 5

port = qubeui.new_port_handler(qube)

ms2038 = instruments.MS2038_SPA(visa)
ms2038.write(':INIT:CONT OFF')

fname = 'Qube{}{}_Port{}.pickle'.format(QUBE, QUBE_ID, PORT_ID)
result = []
# FL = 10400
# FI = 1800
for FL in range(10400, 11001, 100):
    for FI in range(1800,3201,50):
        FR = FL - FI
        qubeui.set_flo_fnco_according_to_frf(port[PORT_ID], FR, FI)
        qubeui.set_vatt(port[PORT_ID].upconv.vatt, 0xA00)
        data = ms2038.measure()
        data['Qube'] = QUBE
        data['Qube ID'] = QUBE_ID
        data['Port ID'] = PORT_ID
        data['VATT'] = 0xA00
        data['IF'] = FI
        data['LO'] = FL
        data['AD9082 Current Mode'] = '(0x01B, 0x0F), (0x117, 0xA0), (0x118, 0xFF)'
        result.append(data)
        p = bokeh.plotting.figure(height=300, toolbar_location=None, sizing_mode="stretch_width")
        p.line(data['numpy_array'][0,:],data['numpy_array'][1,:])
        clear_output()
        bokeh.plotting.show(p)
with open(fname, 'wb') as f:
    pickle.dump(data, f)    